In [1]:
import numpy as np
import pandas as pd
from spacy.lang.en import English
import spacy
from collections import Counter
from math import sqrt
from nltk.corpus import wordnet
import json
#from py_thesaurus import Thesaurus

In [2]:
with open('Interchange File.json') as file:
        interchange = json.load(file)
        
with open('Source File.json') as src:
        source = json.load(src)

In [3]:
error_Msg = "Address is wrong"

In [4]:
data_Dict = pd.read_csv("Data Dictionary.csv")

In [5]:
data_Dict

,Primary Object,Field Name,Field description,Canonical Values
0,Customer,FirstName,First Name of the Applicant,NaN
1,NaN,LastName,Last Name of the Applicant,NaN
2,NaN,AddressLine1,Address Line1 of the Applicant,Address
3,NaN,AddressLine2,Address Line2 of the Applicant,Address
4,NaN,Zipcode,Social Security Number of the Applicant,"pincode, area code"
5,NaN,SSN,Social Security Number of the Applicant,"Social security number, unique id"
6,Claim Details,RevenueCode,Revenue Code for the Claim,NaN
7,NaN,ClaimAmount,Total Claim Amount of the Claim,NaN


### Checking the list of stopwords in spacy

In [6]:
spacy_Stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('Number of stop words: %d' % len(spacy_Stopwords))
print('First ten stop words: %s' % list(spacy_Stopwords)[:10])

Number of stop words: 326
First ten stop words: ['toward', 'three', 'be', 'my', 'below', 'formerly', 'must', 'own', 'himself', 'whole']


### "nlp" Object is used to create documents with linguistic annotations.

In [7]:
nlp = English()

In [8]:
my_msg = nlp(error_Msg)
my_msg

Address is wrong

### Creating list of tokens from the error message

In [9]:
token_list = []
for token in my_msg:
    token_list.append(token.text)

In [10]:
token_list

['Address', 'is', 'wrong']

### Creating list of tokens after removing stopwords


In [11]:
filtered_errorMsg = []
for word in token_list:
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == False:
        filtered_errorMsg.append(word)

In [12]:
print(token_list)

['Address', 'is', 'wrong']


In [13]:
print(filtered_errorMsg)

['Address', 'wrong']


In [14]:
print(" ".join(filtered_errorMsg))

Address wrong


### Lemmatization

In [15]:
filtered_Msg = []
new_myMsg = ""
for word in filtered_errorMsg:
    new_myMsg += word + " "
new_myMsg = nlp(new_myMsg)
new_myMsg

Address wrong 

In [16]:
for lem_word in new_myMsg:
    filtered_Msg.append(lem_word.lemma_)

In [17]:
filtered_Msg

['Address', 'wrong']

In [18]:
print( " ".join(filtered_Msg))

Address wrong


### Similarity between words

In [19]:
data_Dict.columns

Index(['Primary Object', 'Field Name ', 'Field description',
       'Canonical Values'],
      dtype='object')

In [20]:
# removing white space at the end of the column name 
data_Dict.columns = data_Dict.columns.str.rstrip()

In [21]:
data_Dict.columns

Index(['Primary Object', 'Field Name', 'Field description',
       'Canonical Values'],
      dtype='object')

In [22]:
field_Name= data_Dict['Field Name'].to_list()

In [23]:
field_Name

['FirstName ',
 'LastName ',
 'AddressLine1',
 'AddressLine2',
 'Zipcode ',
 'SSN',
 'RevenueCode',
 'ClaimAmount ']

In [25]:
def word2vec(word):
    
    # count the characters in word
    cw = Counter(word)
    
    # set of the different characters
    sw = set(cw)
    
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))
    
    # return a tuple
    return cw, sw, lw

def cosdis(v1, v2):
    # which characters are common to the two words
    common = v1[1].intersection(v2[1])
   
    # by definition of cosine distance
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]

threshold = 0.80    
for key in filtered_Msg:
    for word in field_Name:
        try:

            res = cosdis(word2vec(word), word2vec(key))
            if res > threshold:
                print("Found a matching word {} with original word: {}".format(word, key))
                new_word = word
                print("The similar word from Field names is: ", new_word)
                # Interchange file
                class AppConfiguration():
                    def __init__(self, data=None):
                        if data is None:
                            with open("Interchange File.json") as fh:
                                data = json.loads(fh.read())
                        else:
                            data = dict(data)
                        for key, val in data.items():
                            setattr(self, key, self.compute_attr_value(val))

                    def compute_attr_value(self, value):
                        if type(value) is list:
                            return [self.compute_attr_value(x) for x in value]
                        elif type(value) is dict:
                            return AppConfiguration(value)
                        else:
                            return value
                instance = AppConfiguration()
                custInstance=instance.Customer
                interchange_attribute=getattr(custInstance,new_word)
#               print(a)
#               inter_att = instance.Customer.AddressLine2
                print("The error to be replaced in the interchange file is :  ",interchange_attribute)
                
                class AppConfiguration():
                    def __init__(self, data=None):
                        if data is None:
                            with open("Source File.json") as fh:
                                data = json.loads(fh.read())
                        else:
                            data = dict(data)
                        for key, val in data.items():
                            setattr(self, key, self.compute_attr_value(val))

                    def compute_attr_value(self, value):
                        if type(value) is list:
                            return [self.compute_attr_value(x) for x in value]
                        elif type(value) is dict:
                            return AppConfiguration(value)
                        else:
                            return value
                instance = AppConfiguration()
                custInstance=instance.Customer
                source_attribute=getattr(custInstance,new_word)
                print("The error to be replaced with from the source file is :  ",source_attribute)
                interchange['Customer'][new_word] = source_attribute
                with open('Interchange File.json', 'w') as fp:
                    json.dump(interchange, fp,  indent=4)
                
        except IndexError:
            pass
        
# You can get the name by accessing the _name_ attribute of the class; 
# and if you have an object, you can get the class by accessing the _class_ attribute of the object.

Found a matching word AddressLine1 with original word: Address
The similar word from Field names is:  AddressLine1
The error to be replaced in the interchange file is :   302, Street Avenue
The error to be replaced with from the source file is :   302, Street Avenue
Found a matching word AddressLine2 with original word: Address
The similar word from Field names is:  AddressLine2
The error to be replaced in the interchange file is :   India
The error to be replaced with from the source file is :   japan


In [26]:
### Finding the synonym of the error word

In [1]:
# synonyms = []
# for syn in wordnet.synsets(new_word):
#     for lm in syn.lemmas():
#              synonyms.append(lm.name())#adding into synonyms
# syn_List = list(set(synonyms))
# print (syn_List)

In [57]:
print("Above the function")
def word2vec_syn(word):
    
    # count the characters in word
    cw = Counter(word)
    
    # set of the different characters
    sw = set(cw)
    
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))
#     print("Inside the function")

    # return a tuple
    return cw, sw, lw

def cosdis_syn(v1, v2):
    # which characters are common to the two words
    common = v1[1].intersection(v2[1])
   
    # by definition of cosine distance
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]

threshold = 0.75    
for key in syn_List:
    for word in field_Name:
        try:
#           Cosine simililarity
            res = cosdis_syn(word2vec_syn(word), word2vec_syn(key))
#           print("The cosine similarity between : {} and : {} is: {}".format(word, key, res*100))
            if res > threshold:
                print("Found a matching word {} with original word: {}".format(word, key))
#                 new_word_syn = key
#                 print("The new word is ", new_word_syn)
        except IndexError:
            pass

Above the function
Found a matching word Address Line1 with original word: address
Found a matching word Address Line2 with original word: address


In [59]:
adjSyn = pd.read_csv("ErrorAdjectivesSynonym.txt", sep = '\t')

In [60]:
adjSyn

,Adjectives,Synonyms
0,Invalid,faulty incorrect not-working wrong false defec...
1,Required,Excpected mandatory needed compulsory
2,Match,equal duplicate equivalent
3,missing,incomplete misplaced removed short not-present
